In [ ]:
%%manim -v WARNING -qh DBSCAN
from manim import *
from shapely.geometry import Point
from shapely.ops import unary_union

class DBSCAN(Scene):
    def construct(self):
        # Step 1: 初始显示 "DBSCAN"，直接上移一点
        short_text = Text("DBSCAN", font="Arial", font_size=96, color=BLUE_D)
        short_text.shift(UP * 1)
        self.play(Write(short_text))
        self.wait(1)

        # Step 2: 展开后的完整词组（大写 + 小写）
        parts = [
            ("D", "ensity-"),
            ("B", "ased "),
            ("S", "patial "),
            ("C", "lustering of "),
            ("A", "pplications with "),
            ("N", "oise")
        ]
        expanded = VGroup()
        for cap, ext in parts:
            cap_letter = Text(cap, font="Arial", font_size=96, color=BLUE_D)
            extension = Text(ext, font="Arial", font_size=70, color=BLUE_B)
            extension.next_to(cap_letter, RIGHT, buff=0.1, aligned_edge=ORIGIN)
            extension.shift(DOWN * 0.08)
            if cap in ["D", "S", "C", "A"]:
                extension.shift(DOWN * 0.13)
            group = VGroup(cap_letter, extension)
            if cap == "N":
                cap_letter.shift(UP * 0.05)
                group = VGroup(cap_letter, extension)
                group.shift(UP * 0.7)
            expanded.add(group)
        expanded.arrange(RIGHT, buff=0.1).scale(0.5)
        expanded.move_to(short_text.get_center())

        # Step 3: Transform 动画替换
        animations = []
        fadeins = []
        for i, letter in enumerate(short_text):
            cap_letter, extension = expanded[i]
            animations.append(Transform(letter, cap_letter))
            fadeins.append(FadeIn(extension, shift=LEFT))
        self.play(*animations, *fadeins, run_time=3)
        self.wait(1)

        # Step 4: 显示中文翻译
        full_text = "基于密度的带噪声应用空间聚类算法"
        translation_chars = [Text(c, font="Microsoft YaHei", font_size=50, color=GREEN_C) for c in full_text]
        translation_group = VGroup(*translation_chars).arrange(RIGHT, buff=0.1)
        translation_group.next_to(expanded, DOWN, buff=0.8)
        self.play(Write(translation_group))
        self.wait(2)

        # Step 5: 保留"密度"，其他全部消失并移动到屏幕中心
        start_idx = full_text.index("密度")
        density_chars = VGroup(translation_chars[start_idx], translation_chars[start_idx + 1])
        other_chars = VGroup(*[c for i, c in enumerate(translation_chars) if i not in [start_idx, start_idx + 1]])

        # 英文全称 + DBSCAN + 中文其他字符都加入消失列表
        other_objects = VGroup(expanded, short_text, other_chars)

        # 同时执行消失动画，密度移动到中心
        self.play(
            FadeOut(other_objects),
            density_chars.animate.move_to(ORIGIN).scale(1.8).shift(UP*0.8).set_color(RED),
            run_time=2
        )
        self.wait(2)

        t1 = Text("密度可达", font="Microsoft YaHei", font_size=50, color=ORANGE)
        t2 = Text("密度可连接", font="Microsoft YaHei", font_size=50, color=ORANGE)
        t3 = Text("噪声", font="Microsoft YaHei", font_size=50, color=BLUE_C)
        t1.next_to(density_chars, LEFT, buff=1.0)
        t2.next_to(density_chars, RIGHT, buff=1.0)
        t3.next_to(density_chars, DOWN, buff=1.0)
        self.play(Write(t1), Write(t2))
        self.wait(0.5)
        left_line = Line(start=t1.get_bottom()+DOWN*0.3, end=t3.get_left() + LEFT*0.3, stroke_width=4, color=WHITE)
        left_start_dot = Dot(point=t1.get_bottom()+DOWN*0.3, radius=0.1, color=WHITE)
        left_end_dot = Dot(point=t3.get_left() + LEFT*0.3, radius=0.1, color=WHITE)

        right_line = Line(start=t2.get_bottom()+DOWN*0.3, end=t3.get_right() + RIGHT*0.3, stroke_width=4, color=WHITE)
        right_start_dot = Dot(point=t2.get_bottom()+DOWN*0.3, radius=0.1, color=WHITE)
        right_end_dot = Dot(point=t3.get_right() + RIGHT*0.3, radius=0.1, color=WHITE)
        self.play(
            Create(left_line), Create(left_start_dot), Create(left_end_dot),
            Create(right_line), Create(right_start_dot), Create(right_end_dot)
        )
        self.wait(0.5)
        self.play(Write(t3))
        self.wait(2)

        # 全屏黑色覆盖
        black_screen = Rectangle(
            width=self.camera.frame_width,
            height=self.camera.frame_height,
            fill_color=BLACK,
            fill_opacity=1.0,
            stroke_width=0
        )
        self.play(FadeIn(black_screen))  # 直接添加黑屏
        self.wait(3)


        title = Text("密度可达", font="Microsoft YaHei", font_size=60, color=BLUE)
        title.to_edge(UP)
        self.add(title)
        # 坐标轴
        axes = Axes(
            x_range=[0, 4, 1],
            y_range=[0, 4, 1],
            x_length=5.5,
            y_length=5.5,
            axis_config={"color": WHITE}
        )
        axes.next_to(title, DOWN, buff=0.6)
        # 数据点（全部放在一个列表里，不分可达或噪声）
        points = np.array([
            [1.2, 2.0],
            [2.3, 2.35],
            [2.7, 2.8],
            [2.9, 2.3],
            [3.05, 1.9],
            [3.2, 1.35],
            [3.5, 3.5],
        ])
        points[:, 0] -= 0.5  # x 统一平移
        points[:, 1] -= 0.7  # y 统一平移
        # 创建 VGroup
        point_list = VGroup()
        for p in points:
            dot = Dot(point=axes.c2p(p[0], p[1]), radius=0.08, color=WHITE)
            point_list.add(dot)
        # FadeIn 坐标轴和点
        self.play(FadeIn(axes), FadeIn(point_list))
        self.wait(2)

        # Step: 邻域半径
        epsilon_text = Text("(epsilon)ε=0.9", font="Microsoft YaHei", font_size=35, color=GREEN_C)
        epsilon_text.next_to(axes, LEFT, buff=0.5).shift(UP*0.8)
        self.play(Write(epsilon_text))

        # Step: 以第一个点为中心画圆
        center_point = points[0]
        epsilon_radius = 0.9  # ε
        circle = Circle(radius=epsilon_radius, color=WHITE, fill_opacity=0.2)
        circle.move_to(axes.c2p(center_point[0], center_point[1]))
        self.play(Create(circle))
        self.wait(2)
        # 第一个圆消失
        self.play(FadeOut(circle))
        self.wait(0.5)
        # 第二个圆
        center_point2 = points[1]
        circle2 = Circle(radius=epsilon_radius, color=WHITE, fill_opacity=0.2)
        circle2.move_to(axes.c2p(center_point2[0], center_point2[1]))
        self.play(Create(circle2))
        self.wait(2)
        # 第2、3、4个点变红
        for i in [1, 2, 3]:
            dot_list = point_list[i]  # VGroup 中的 Dot
            self.play(dot_list.animate.set_color(RED))
        self.wait(1)
        for i in [0, 4, 5, 6]:
            dot_list = point_list[i]  # VGroup 中的 Dot
            self.play(dot_list.animate.set_color(BLUE))
        self.wait(4)

        # -------- 新增：第3个红点画圆 --------
        center_point3 = points[2]
        circle3 = Circle(radius=epsilon_radius, color=WHITE, fill_opacity=0.2)
        circle3.move_to(axes.c2p(center_point3[0], center_point3[1]))
        self.play(Create(circle3))
        # -------- 新增：第4个红点画圆 --------
        center_point4 = points[3]
        circle4 = Circle(radius=epsilon_radius, color=WHITE, fill_opacity=0.2)
        circle4.move_to(axes.c2p(center_point4[0], center_point4[1]))
        self.play(Create(circle4))
        self.wait(2)
        # -------- 新增：第5个点变红 --------
        self.play(point_list[4].animate.set_color(RED))
        self.wait(1)
        # -------- 新增：第5个红点画圆 --------
        center_point5 = points[4]
        circle5 = Circle(radius=epsilon_radius, color=WHITE, fill_opacity=0.2)
        circle5.move_to(axes.c2p(center_point5[0], center_point5[1]))
        self.play(Create(circle5))
        # -------- 新增：第6个点变红 --------
        self.play(point_list[5].animate.set_color(RED))
        self.wait(1)
        # -------- 新增：第6个红点画圆 --------
        center_point6 = points[5]
        circle6 = Circle(radius=epsilon_radius, color=WHITE, fill_opacity=0.2)
        circle6.move_to(axes.c2p(center_point6[0], center_point6[1]))
        self.play(Create(circle6))
        self.wait(2)
        # -------- 计算所有圆的并集外轮廓并用红色描边和淡红填充 --------
        shapely_circles = []
        circle_mobs = [circle2, circle3, circle4, circle5, circle6]
        for i in range(1, 6):  # 2~6号点
            cx, cy = points[i]
            shapely_circles.append(Point(cx, cy).buffer(0.7, resolution=512))
        union_shape = unary_union(shapely_circles)
        boundary_group = VGroup()
        if union_shape.geom_type == "Polygon":
            polys = [union_shape]
        else:
            polys = union_shape.geoms
        for poly in polys:
            coords = np.array(poly.exterior.coords)
            manim_pts = [axes.c2p(x, y) for x, y in coords]
            path = VMobject(
                stroke_color=RED,       # 红色轮廓
                stroke_width=2,
                fill_color=RED,         # 淡红填充
                fill_opacity=0.2
            )
            path.set_points_smoothly(manim_pts)
            boundary_group.add(path)
        # 同时执行：轮廓出现 + 圆消失
        self.play(
            Create(boundary_group),
            *[FadeOut(circ) for circ in circle_mobs],
            run_time=2
        )
        self.wait(2)
        # 假设 title2 已经存在，文字是"密度可达"
        title2 = Text("密度可连接", font="Microsoft YaHei", font_size=60, color=BLUE)
        title2.to_edge(UP)
        # 用 Transform 动画替换原来的 title2
        self.play(Transform(title, title2))
        self.wait(2)
        # 在坐标轴右边添加 "MinPts"
        minpts_text = Text("MinPts=4", font="Microsoft YaHei", font_size=40, color=GREEN_C)
        minpts_text.next_to(epsilon_text, DOWN, buff=0.9)
        self.play(Write(minpts_text))
        self.wait(2)
        epsilon_radius = 0.94  # 半径
        previous_circle = None
        for i in range(1, 6):  # points[1]~points[5]
            circle = Circle(radius=epsilon_radius, color=RED, stroke_width=3)
            circle.move_to(axes.c2p(points[i][0], points[i][1]))
            if previous_circle is None:
                self.play(Create(circle), run_time=1)
            else:
                self.play(Create(circle), FadeOut(previous_circle), run_time=1)
            previous_circle = circle
            if i == 3:
                self.wait(2)
                self.play(point_list[3].animate.set_color(YELLOW))
            else:
                self.wait(0.8)
        self.play(FadeOut(previous_circle))
        self.play(point_list[5].animate.set_color(BLUE))
        self.wait(2)

        # 核心点：第四个点 points[3]
        core_start = axes.c2p(points[3][0]+0.1, points[3][1])  # 点右边一点
        core_end = axes.c2p(points[3][0]+1.8, points[3][1])    # 向右延伸
        core_arrow = Arrow(start=core_start, end=core_end, color=YELLOW, buff=0)
        core_text = Text("核心点", font="Microsoft YaHei", font_size=36, color=YELLOW)
        core_text.next_to(core_arrow, RIGHT, buff=0.2)
        # 边缘点：第五个点 points[4]
        edge_start = axes.c2p(points[4][0]+0.1, points[4][1])
        edge_end = axes.c2p(points[4][0]+1.8, points[4][1])
        edge_arrow = Arrow(start=edge_start, end=edge_end, color=RED, buff=0)
        edge_text = Text("边缘点", font="Microsoft YaHei", font_size=36, color=RED)
        edge_text.next_to(edge_arrow, RIGHT, buff=0.2)
        # 噪声点：最后一个点 points[6]
        noise_start = axes.c2p(points[6][0]+0.1, points[6][1])
        noise_end = axes.c2p(points[6][0]+1.8, points[6][1])
        noise_arrow = Arrow(start=noise_start, end=noise_end, color=BLUE, buff=0)
        noise_text = Text("噪声点", font="Microsoft YaHei", font_size=36, color=BLUE)
        noise_text.next_to(noise_arrow, RIGHT, buff=0.2)
        # 噪声点2：点 points[5]
        noise_start2 = axes.c2p(points[5][0]+0.1, points[5][1])
        noise_end2 = axes.c2p(points[5][0]+1.8, points[5][1])
        noise_arrow2 = Arrow(start=noise_start2, end=noise_end2, color=BLUE, buff=0)
        noise_text2 = Text("噪声点", font="Microsoft YaHei", font_size=36, color=BLUE)
        noise_text2.next_to(noise_arrow2, RIGHT, buff=0.2)
        # 播放动画
        self.play(Create(core_arrow), Write(core_text))
        self.wait(2)
        self.play(Create(edge_arrow), Write(edge_text))
        self.wait(2)
        self.play(Create(noise_arrow), Write(noise_text))
        self.wait(3)
        self.play(Create(noise_arrow2), Write(noise_text2))
        self.wait(5)

        # epsilon tracker
        epsilon_tracker = ValueTracker(0.7)  # 初始半径和你之前的 buffer 一致
        # Step: 半径变化前，先让原来的 epsilon_text 消失
        self.play(FadeOut(epsilon_text), run_time=0.5)

        # 创建新的 epsilon_text 固定部分
        epsilon_text_dynamic = Text("(epsilon)ε=", font="Microsoft YaHei", font_size=35, color=GREEN_C)
        epsilon_text_dynamic.move_to(epsilon_text.get_center())
        self.add(epsilon_text_dynamic)

        # DecimalNumber 动画（同步显示 epsilon）
        epsilon_value = DecimalNumber(
            epsilon_tracker.get_value(),
            num_decimal_places=2,
            font_size=50,
            color=GREEN_C
        )
        epsilon_value.next_to(epsilon_text_dynamic, RIGHT, buff=0.05)
        self.add(epsilon_value)

        # updater：每帧同步更新数值
        def update_epsilon_value(mob):
            mob.set_value(epsilon_tracker.get_value())
            return mob
        epsilon_value.add_updater(update_epsilon_value)

        # 更新函数：直接让 boundary_group 变形
        def update_boundary(boundary_mob):
            r = epsilon_tracker.get_value()
            # 根据 r 重建 shapely 并集
            shapely_circles = [Point(points[i+1][0], points[i+1][1]).buffer(r, resolution=128) for i in range(5)]
            union_shape = unary_union(shapely_circles)

            new_boundary = VGroup()
            if union_shape.geom_type == "Polygon":
                polys = [union_shape]
            else:
                polys = union_shape.geoms
            for poly in polys:
                coords = np.array(poly.exterior.coords)
                pts = [axes.c2p(x, y) for x, y in coords]
                path = VMobject(stroke_color=RED, stroke_width=2, fill_color=RED, fill_opacity=0.2)
                path.set_points_smoothly(pts)
                new_boundary.add(path)
            
            boundary_mob.become(new_boundary)
            return boundary_mob
        # 给已有轮廓添加 updater
        boundary_group.add_updater(update_boundary)
        # 开始半径动画
        self.play(epsilon_tracker.animate.set_value(1.2), run_time=3)
        self.wait(2)
        self.play(epsilon_tracker.animate.set_value(0.1), run_time=4)
        self.wait(2)
        # 半径恢复到 0.7
        self.play(epsilon_tracker.animate.set_value(0.7), run_time=2)
        self.wait(3)
        # 移除 updaters
        boundary_group.clear_updaters()
        epsilon_value.clear_updaters()